In [23]:
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
# from sklearn.datasets import fetch_20newsgroups

#not required in my case ...n_samples = 30
n_features = 50
n_components = 6 # topics
n_top_words = 8 # words in each topic

In [24]:
%time
# dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=("headers", "footers", "quotes"))
# datasamples = dataset.data[:n_samples]


## own data from the amazon reviews

import pandas as pd

# location of the data
# datafile = "../data/amazon5core/digitalmusic/digitalmusic5core.csv"
datafile = "../data/amazon5core/healthcare/healthcare5core.csv"
# datafile = "../data/amazon5core/cellphone/cellphone5core.csv"


data = pd.read_csv(datafile, dtype={"asin": str, "helpful": str, "reviewText": str, "reviewerID": str})

data.head

Wall time: 0 ns


<bound method NDFrame.head of               asin      reviewerID  overall  \
0       B002T3DBKW  A339O8ZW72WHZ0        5   
1       B001J5H92C   A8QC2O7KA1V8D        5   
2       B00EDR6YB0  A2PZ39BYJ4C1LS        5   
3       B004OZGG0K   A4P3D4248880B        4   
4       B000BABW5Q  A26MQ4YGVS5CC2        5   
5       B008BQI8YA   AUP0UHW4TVP1D        3   
6       B003V32UDI  A19DW3VKM5KKYB        5   
7       B00K5NEORM   AX0ZTNX6KVJ40        5   
8       B0014YTACK  A2F69JAZ214QDS        4   
9       B00GBQ3K54   AJOZL929DDQPL        4   
10      B000NL0T1G  A1R4U9YQRE685X        5   
11      B004H4B3VK  A10FYJ0D5Q623M        4   
12      B00C7DIFUA  A3VJ27010XUWTF        5   
13      B00DP5VX58  A2QRXQPHDMFCQV        4   
14      B0014ATBRI  A297R44X7WEQ8T        4   
15      B0051U7R0A   AULDDIOHU5I60        2   
16      B0013ITH7U  A31CUOIOLF6VVJ        4   
17      B007P7AMAO  A32NEDX6UVV8DE        5   
18      B00028LZ90  A1KVXT3C21ZEFL        5   
19      B003X5FYJQ   AK7CAW3N1

In [25]:
## moulding the data to fit the LDA. 
# we will not find all the reviews for the item specific B00LOMZDM8

item_id1 = "B008HPRQ3Y"

# for rows
row_data = data.loc[data['asin'] == item_id1]
# for columns
review_data = row_data.iloc[:,3]
## found 15 reviews for the item specific B00LOMZDM8

# datatext = ""
datatext = review_data.tolist()

#ratings on the same product
rating_data = row_data.iloc[:,2]

rating_data = rating_data.tolist()


# for review in data:
#     datatext += review+". "

print(datatext)

print(rating_data)
data.count()

['Works just like it should, easy to read, very helpful.Definitely recommend this app to keep track of your activity and sleep.', 'What an awesome motivational device.  I find myself going out of my way to walk up stairs and often take the long way home just to achieve higher levels.  I compete with friends and am losing weight while building stamina.', "It's incentive ~ it's amazing how a little instant feedback can be the incentive you need to walk those extra few blocks.  Fitbit is not just for the athletes out there.  Fitbit is easy to use and is packed with great features that help you accomplish your goals, even if they are as simple as just walking more", 'I installed the app on my Kindle Fire and on my Android HTC Rezound.  I have never seen stats appear on either, yet they both report "last synced on...{date}". Everything is showing as "0".  I logged in fine as far as I know. It "knows" my name. The circle was spinning forever and I left it alone. Later I saw it had logged in.

asin          346355
reviewerID    346355
overall       346355
reviewText    346307
dtype: int64

In [26]:
# tfidf vectorizer 

%time

# max_features is the key for the matrix vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(datatext)

Wall time: 0 ns


In [27]:
# tf vectorizer used for LDA - topic Modelling

%time
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(datatext)

Wall time: 0 ns


In [28]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=25, learning_method='online', learning_offset=50, random_state=0)

lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=25, mean_change_tol=0.001,
             n_components=6, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [29]:
# using visualisation for the lda
import pyLDAvis
import pyLDAvis.sklearn

tree = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer) # there are different variations of this function.

pyLDAvis.display(tree)


In [30]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [31]:
tf_feature_names = tf_vectorizer.get_feature_names()

print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: computer calories eat burned weight good steps tells
Topic #1: fitbit app sleep just really great need does
Topic #2: fitbit lost device good track read far kindle
Topic #3: day track fitbit easy computer sleep great like
Topic #4: stairs walk weight just device read sleep help
Topic #5: calories fitbit active app steps don burned help

